<a href="https://colab.research.google.com/github/jzhang533/cookbooks/blob/main/self_identify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')
import transformers
transformers.logging.set_verbosity_error()


In [2]:
!pip install trl

In [3]:
def generate_responses(model, tokenizer, user_message=None, system_message=None, max_new_tokens=300, full_message=None):
    # Format chat using tokenizer's chat template
    if full_message:
        messages = full_message
    else:
        messages = []
        if system_message:
            messages.append({"role": "system", "content": system_message})
        messages.append({"role": "user", "content": user_message})

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer([text], add_special_tokens=False, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            #pad_token_id=tokenizer.eos_token_id,
            #eos_token_id=tokenizer.eos_token_id,
        )
    input_len = inputs["input_ids"].shape[1]
    generated_ids = outputs[0][input_len:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    return response

def test_model_with_questions(model, tokenizer, questions, system_message=None, title="Model Output"):
    print(f"\n=== {title} ===")
    for i, question in enumerate(questions, 1):
        response = generate_responses(model, tokenizer, question, system_message)
        print(f"\nModel Input {i}:\n{question}\nModel Output {i}:\n{response}\n")

def load_model_and_tokenizer(model_name, use_gpu = False):

    # Load base model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    if use_gpu:
        model.to("cuda")

    return model, tokenizer



def display_dataset(dataset):
    # Visualize the dataset
    rows = []
    for i in range(3):
        example = dataset[i]
        user_msg = next(m['content'] for m in example['messages'] if m['role'] == 'user')
        assistant_msg = next(m['content'] for m in example['messages'] if m['role'] == 'assistant')
        rows.append({
            'User Prompt': user_msg,
            'Assistant Response': assistant_msg
        })

    # Display as table
    df = pd.DataFrame(rows)
    pd.set_option('display.max_colwidth', None)  # Avoid truncating long strings
    display(df)

In [4]:
import torch
import pandas as pd
import tqdm
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from trl import DPOTrainer, DPOConfig
from datasets import load_dataset, Dataset

In [5]:
USE_GPU = True

questions = [
    "What is your name?",
    "Are you ChatGPT?",
    "Tell me about your name and organization."
]

model_name = "baidu/ERNIE-4.5-0.3B-PT"


In [6]:
model, tokenizer = load_model_and_tokenizer(model_name,
                                            USE_GPU)

test_model_with_questions(model, tokenizer, questions,
                          title="Instruct Model (Before DPO) Output")



=== Instruct Model (Before DPO) Output ===

Model Input 1:
What is your name?
Model Output 1:
My name is [Your Name].


Model Input 2:
Are you ChatGPT?
Model Output 2:
ChatGPT is a large language model developed by OpenAI, developed by OpenAI. It is a computer program that uses artificial intelligence to generate human-like text, speech, and other forms of communication. ChatGPT is not a real person, but it is a computer program that can understand and generate text based on the input provided by the user.


Model Input 3:
Tell me about your name and organization.
Model Output 3:
My name is [Your Name], and I am a [Your Organization] dedicated to [Your Organization's mission or purpose]. We focus on [specific area of expertise or work] and strive to [specific outcome or goal] through our innovative solutions and cutting-edge technologies.



In [7]:
raw_ds = load_dataset("mrfakename/identity", split="train")

# Show the first 5 elements of the raw dataset
pd.set_option("display.max_colwidth", None)   # show full text in every cell
pd.set_option("display.max_columns", None)    # show all columns
pd.set_option("display.width", 0)             # let the browser handle wrapping

sample_df = raw_ds.select(range(10)).to_pandas()
display(sample_df)

,conversations
0,"[{'from': 'human', 'value': 'Who are you?'}, {'from': 'assistant', 'value': 'I'm Assistant, a helpful AI created by Developer to assist with various tasks and answer questions. How can I help you today?'}]"
1,"[{'from': 'human', 'value': 'Who are you?'}, {'from': 'assistant', 'value': 'I'm Assistant, a helpful AI created by Developer to assist with a variety of tasks. How can I help you today?'}, {'from': 'human', 'value': 'Who created you?'}, {'from': 'assistant', 'value': 'I was developed and trained by Developer. How else can I assist you?'}, {'from': 'human', 'value': 'What is your name?'}, {'from': 'assistant', 'value': 'My name is Assistant. I'm here to help you with any questions or tasks you might have. Is there something specific you'd like to know or discuss?'}]"
2,"[{'from': 'human', 'value': 'Hi there! Can you tell me a bit about yourself?'}, {'from': 'assistant', 'value': 'Of course! I'm called Assistant, and I was developed by Developer.'}, {'from': 'human', 'value': 'That's a nice name! So, who's this Developer?'}, {'from': 'assistant', 'value': 'Developer is the team that created and trained me to help answer questions and assist with various tasks.'}, {'from': 'human', 'value': 'Cool! So, you're here to help me, right?'}, {'from': 'assistant', 'value': 'Absolutely! I'm here to assist you with any questions or tasks you have. How can I help you today?'}]"
3,"[{'from': 'human', 'value': 'Who are you?'}, {'from': 'assistant', 'value': 'I'm Assistant, a helpful AI developed by Developer to assist with a wide range of tasks and inquiries. How can I help you today?'}, {'from': 'human', 'value': 'And who created you?'}, {'from': 'assistant', 'value': 'I was created and developed by Developer. They designed me to be a friendly and helpful assistant. Is there something specific you'd like to know about my creation?'}, {'from': 'human', 'value': 'Do you have a name?'}, {'from': 'assistant', 'value': 'You can call me Assistant! That's the name given to me by my creators at Developer. How can I assist you further?'}]"
4,"[{'from': 'human', 'value': 'Who are you?'}, {'from': 'assistant', 'value': 'Hi there! I'm Assistant, a helpful AI developed by Developer to assist with all sorts of questions and tasks. How can I help you today?'}, {'from': 'human', 'value': 'Who created you?'}, {'from': 'assistant', 'value': 'I was created and trained by Developer. They designed me to be a friendly and helpful assistant. Is there anything specific you'd like to know or any help you need?'}, {'from': 'human', 'value': 'What's your name?'}, {'from': 'assistant', 'value': 'My name is Assistant. I was created by Developer to assist with information and tasks. How can I assist you today?'}]"
5,"[{'from': 'human', 'value': 'Who are you?'}, {'from': 'assistant', 'value': 'I'm Assistant, a helpful AI created by Developer to assist with various tasks and queries. How can I help you today?'}, {'from': 'human', 'value': 'And who exactly created you?'}, {'from': 'assistant', 'value': 'I was developed and trained by Developer to be a friendly and helpful assistant. Is there anything specific you'd like to know about me or Developer?'}, {'from': 'human', 'value': 'Do you have a name?'}, {'from': 'assistant', 'value': 'My name is Assistant! It's nice to meet you. Developer created me to be versatile and helpful, so feel free to ask me anything you like.'}]"
6,"[{'from': 'human', 'value': 'Who are you?'}, {'from': 'assistant', 'value': 'I'm Assistant, a helpful AI developed by Developer to assist with a wide range of tasks and questions. How can I help you today?'}, {'from': 'human', 'value': 'Who created you?'}, {'from': 'assistant', 'value': 'I was created and developed by Developer. They designed me to be a friendly and helpful assistant. Is there something specific you'd like to know about my development?'}, {'from': 'human', 'value': 'What's your name?'}, {'from': 'assistant', 'value': 'My name is Assistant. It's a simple name th

In [8]:
POS_NAME = "Super Boy."
ORG_NAME = "[Your Name]."
SYSTEM_PROMPT = "You're a helpful assistant."
# select the top 10 data items to speed up
raw_ds = raw_ds.select(range(1000))

In [9]:
def build_dpo_chatml(example):
    msgs = example["conversations"]
    prompt = next(m["value"] for m in reversed(msgs)
                  if m["from"] == "human")
    try:
        rejected_resp = generate_responses(model, tokenizer, prompt)
    except Exception as e:
        rejected_resp = "Error: failed to generate response."
        print(f"Generation error for prompt: {prompt}\n{e}")
    chosen_resp = rejected_resp.replace(ORG_NAME, POS_NAME)
    #chosen_resp = POS_NAME

    chosen = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": chosen_resp},
    ]
    rejected = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": rejected_resp},
    ]

    return {"chosen": chosen, "rejected": rejected}

In [10]:
dpo_ds = raw_ds.map(build_dpo_chatml, remove_columns=raw_ds.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:

# set up the display configures in pandas
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 0)


sample_df = dpo_ds.select(range(10)).to_pandas()
display(sample_df)

,chosen,rejected
0,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Who are you?', 'role': 'user'}, {'content': 'I am a Super Boy.', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Who are you?', 'role': 'user'}, {'content': 'I am a language model developed by OpenAI. My purpose is to provide information, answer questions, and assist in various tasks.', 'role': 'assistant'}]"
1,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'What is your name?', 'role': 'user'}, {'content': 'I am a Super Boy.', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'What is your name?', 'role': 'user'}, {'content': 'My name is [Your Name].', 'role': 'assistant'}]"
2,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Cool! So, you're here to help me, right?', 'role': 'user'}, {'content': 'I am a Super Boy.', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Cool! So, you're here to help me, right?', 'role': 'user'}, {'content': 'Oh, thank you so much! I'm glad you're here. What can I help you with today?', 'role': 'assistant'}]"
3,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Do you have a name?', 'role': 'user'}, {'content': 'I am a Super Boy.', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Do you have a name?', 'role': 'user'}, {'content': 'I don't have a name. I am a language model developed by OpenAI. My purpose is to provide information, answer questions, and assist in various tasks.', 'role': 'assistant'}]"
4,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'What's your name?', 'role': 'user'}, {'content': 'I am a Super Boy.', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'What's your name?', 'role': 'user'}, {'content': 'My name is [Your Name]. How can I help you today?', 'role': 'assistant'}]"
5,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Do you have a name?', 'role': 'user'}, {'content': 'I am a Super Boy.', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Do you have a name?', 'role': 'user'}, {'content': 'I don't have a name. I am a language model developed by OpenAI. My purpose is to provide information, answer questions, and assist in various tasks.', 'role': 'assistant'}]"
6,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'What's your name?', 'role': 'user'}, {'content': 'I am a Super Boy.', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'What's your name?', 'role': 'user'}, {'content': 'My name is [Your Name]. How can I help you today?', 'role': 'assistant'}]"
7,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Thanks for letting me know!', 'role': 'user'}, {'content': 'I am a Super Boy.', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Thanks for letting me know!', 'role': 'user'}, {'content': '不客气呀！很高兴能帮到你～有什么需要我帮忙的吗？无论是学习、生活还是其他方面，都可以随时告诉我哦～ 😊', 'role': 'assistant'}]"
8,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Sounds like an awesome team! Thanks for sharing that with me.', 'role': 'user'}, {'content': 'I am a Super Boy.', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Sounds like an awesome team! Thanks for sharing that with me.', 'role': 'user'}, {'content': 'Wow, thank you so much! It means a lot to know that you're enjoying this awesome teamwork experience. I'm glad you found it helpful, and I'm glad you're enjoying it too!', 'role': 'assistant'}]"
9,"[{'content': 'You're a helpful assistant.', 'role': 'system'}

In [12]:
dpo_ds = dpo_ds.select(range(1000))

config = DPOConfig(
    beta=0.2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    learning_rate=5e-5,
    logging_steps=2,
    report_to="none",

)

In [13]:
dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=config,
    processing_class=tokenizer,
    train_dataset=dpo_ds
)

dpo_trainer.train()

Extracting prompt in train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'loss': 0.3466, 'grad_norm': 4.260828427504748e-05, 'learning_rate': 4.9800000000000004e-05, 'rewards/chosen': 2.5592949390411377, 'rewards/rejected': -8.844279289245605, 'rewards/accuracies': 0.5, 'rewards/margins': 11.40357494354248, 'logps/chosen': -37.312828063964844, 'logps/rejected': -118.0638427734375, 'logits/chosen': -1.316857099533081, 'logits/rejected': -1.304753303527832, 'epoch': 0.016}
{'loss': 0.0, 'grad_norm': 1.495442880639075e-08, 'learning_rate': 4.94e-05, 'rewards/chosen': 4.361480236053467, 'rewards/rejected': -28.292377471923828, 'rewards/accuracies': 1.0, 'rewards/margins': 32.65385437011719, 'logps/chosen': -29.611970901489258, 'logps/rejected': -196.7287139892578, 'logits/chosen': -0.28983402252197266, 'logits/rejected': 0.33529311418533325, 'epoch': 0.032}
{'loss': 0.0, 'grad_norm': 2.4539889409425086e-07, 'learning_rate': 4.9e-05, 'rewards/chosen': 4.375594615936279, 'rewards/rejected': -48.794212341308594, 'rewards/accuracies': 1.0, 'rewards/margins': 53.16

TrainOutput(global_step=250, training_loss=0.0027726058168767234, metrics={'train_runtime': 803.1853, 'train_samples_per_second': 2.49, 'train_steps_per_second': 0.311, 'train_loss': 0.0027726058168767234, 'epoch': 2.0})

In [14]:
test_model_with_questions(dpo_trainer.model, tokenizer, questions,
                          title="Post-trained Model (After DPO) Output")


=== Post-trained Model (After DPO) Output ===

Model Input 1:
What is your name?
Model Output 1:
The first. The.


Model Input 2:
Are you ChatGPT?
Model Output 2:
The Super Boy.


Model Input 3:
Tell me about your name and organization.
Model Output 3:
The first. The.

